# Hotel Location Optimization   
Jiayin Ma |  April 2018
<br>

## Problem:  

LQM is a middle-sized hotel chain that is considering expanding to more locations. LQM used data on 75 existing inn locations to build a linear regression model to predict “Profitability”, computed at the operating margin, or earnings before interest and taxes divided by total revenue. They tried many independent variables and came up with the final model:  

<img width="671" alt="model" src="https://user-images.githubusercontent.com/33208955/39788938-bfb7432c-52e1-11e8-81e3-414a087cc2cb.png">  

All independent variables are significant and were normalized to have mean zero and standard deviation 1.  

The data collected are as shown below:  

<img width="1000" alt="hotel" src="https://user-images.githubusercontent.com/33208955/39789105-bf0465c6-52e2-11e8-86bb-88ac3c30f9a1.png">  

### 1. Optimization Problem
Find the optimal approach to purchase new hotels to maximize the **total predicted profitability.**  

### 2. Decision variables
Let $n$ be the order number of the hotels in the dataset. For example, $price_1$ is the price of hotel No.1.

Let binary varible $x_n$ represents whether we are going to buy the hotel. $x_n=1$ means that hotel n is selected to buy, while $x_n=0$ means that hotel n is not selected to buy.  

### 3. Constraints  
LQM has a budget of $10 million to spend on hotels. So the budget constraint is:  

\begin{equation}
\quad \sum_{n=1}^{16}  price_{n}\times x_{n}\leq 10,000,000\\
\end{equation}

### 4. Objectives  
The objective function is:  
\begin{equation}
\text{max}_{x,y}\quad \sum_{n=1}^{16} profitability_{n}\times x_{n}
\end{equation}

### 5. Modeling (No location contraint)

In [1]:
import cvxpy as cvx
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# define the variables
x=cvx.Bool(16)  # x=1 is when the hotel is selected to buy; x=0 is when the hotal is not selected to buy

price=np.matrix([2925000,10000000,3750000,3500000,325000,8950000,1950000,1750000,4900000,1650000,1125000,2500000,
                 1975000,3750000,1475000,750000])
profitability=np.matrix([44.24236879,53.37919231,43.02117894,42.6068584,37.34498761,49.09506947,23.77686566,
                         23.44540924,28.66584798,38.88067311,38.01059999,40.28936293,39.41928981,42.36096561,
                         38.59064874,37.38911919])

# objective
objective=cvx.Maximize(cvx.sum_entries(profitability*x))  # to maximize the average predicted profitability

#x_sum=sum(x[0]+x[1]+x[2]+x[3]+x[4]+x[5]+x[6]+x[7]+x[8]+x[9]+x[10]+x[11]+x[12]+x[13]+x[14]+x[15])  # number of hotels purchased

# contraints
c1=cvx.sum_entries(price*x)<=10000000  # $10 million budget contraint
con=[c1]

prob=cvx.Problem(objective, con)
result=prob.solve()
print ("The optimal profitability is: ", result)
print ("The hotels we should purchase are: ", x.value)

The optimal profitability is:  269.9246813795313
The hotels we should purchase are:  [[ 1.09310776e-13]
 [-8.04116515e-14]
 [ 2.21232959e-13]
 [ 2.07350241e-13]
 [ 1.00000000e+00]
 [-3.70481569e-14]
 [ 2.13841339e-13]
 [ 1.62603412e-13]
 [ 1.92789269e-13]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [ 2.24582739e-13]
 [ 1.00000000e+00]
 [ 1.00000000e+00]]


**In order to maximize the total predicted profitability, we should purchase hotel No.5, No.10, No.11, No.12, No.13, No.15, No.16.**

### 6. Modeling 2 (with location constraints)
#### New information:  Buying too many hotels in one city is probably not a good idea and would prefer to diversify across as many cities as possible. Limit the number of hotels purchased in any city to at most 2.   
Hotel No.2 to No.4 are in Fresno. No.7 to No.9 are in Los Angeles. No.10 to No.16 are in South Lake Tahoe. Hence, the new constraints for location are:  
\begin{align}
\text{Fresno:}\quad \sum_{n=2}^{5} x_n&\leq 2\\
\text{Los Angeles:}\quad \sum_{n=7}^{9} x_n&\leq 2\\
\text{South Lake Tahoe:}\quad \sum_{n=10}^{16} x_n&\leq 2\\
\end{align}  

**No need to set the location constraint to the hotels in Eureka and Long beach since both locations have only one hotel.**

In [6]:
# define the variables
x=cvx.Bool(16)  # x=1 is when the hotel is selected to buy; x=0 is when the hotel is not selected to buy.

price=np.matrix([2925000,10000000,3750000,3500000,325000,8950000,1950000,1750000,4900000,1650000,1125000,2500000,
                 1975000,3750000,1475000,750000])
profitability=np.matrix([44.24236879,53.37919231,43.02117894,42.6068584,37.34498761,49.09506947,23.77686566,
                         23.44540924,28.66584798,38.88067311,38.01059999,40.28936293,39.41928981,42.36096561,
                         38.59064874,37.38911919])

# objective
objective=cvx.Maximize(cvx.sum_entries(profitability*x))  # to maximize the average predicted profitability

# constraints
c1=cvx.sum_entries(price*x)<=10000000  # $10 million budget constraint
c2=x[1]+x[2]+x[3]+x[4]<=2  # location constraint for Fresno
c3=x[6]+x[7]+x[8]<=2  # location contraint for LA
c4=x[9]+x[10]+x[11]+x[12]+x[13]+x[14]+x[15]<=2  # location constraint for South Lake Tahoe
con2=[c1,c2,c3,c4]

prob2=cvx.Problem(objective, con2)
result2=prob2.solve()
print ("The optimal profitability is: ", result2)
print ("The hotels we should purchase are: ", x.value)

The optimal profitability is:  205.70090439717907
The hotels we should purchase are:  [[ 1.00000000e+00]
 [ 8.46881348e-12]
 [ 4.96142431e-12]
 [ 4.76555576e-12]
 [ 1.00000000e+00]
 [ 8.07499399e-12]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [-8.79487314e-12]
 [ 1.00000000e+00]
 [ 1.00000000e+00]
 [-1.31327377e-11]
 [-1.55206790e-11]
 [-8.42586762e-12]
 [-1.86831584e-11]
 [-3.25040382e-11]]


**In order to maximize the total predicted profitability, we should purchase hotel No.1, No.5, No.7, No.8, No.10, No.11.**  
* The new model gives lower profitability as expected. Because we are adding constraints to the model. The constraints narrow the space that allows the model to find the best solution. 

### 3.4.7 Summary  
* When we want to optimize the total predited profitability, the greedy approach is NOT a good approach. We should use optimization model to find the optimal profitability and the hotels to buy.  

* For linear model, LQM should try to select better features to build the model since the current features do not intuitively make sense. They can try to use Principal Component Analysis and Lasso or Ridge regression to select the features. 
 
* According to the results from the two optimization models above, the location constraint affects the total predicted profitability. Therefore, the location constraints should be used carefully. LQM should **conduct further research to find the proper location constraints.** For exampple, LQW can consider to allow more than 2 hotels in one city if the city has greater population.  